In [1]:
import knex

text = """
René Dubois naît le 17.8.1905 aux Verrières, et meurt le 16.11.1976 à Marin-Epagnier, protestant, de Buttes.
Fils d'Henri Adolphe, fonctionnaire postal.
Marié(e) à Suzanne Mina Arnoux, Française, fille de Jules Cyprien.
Licence en sciences économiques à Neuchâtel.
Officier instructeur dès 1929.
Chef d'état-major des troupes d'aviation et de DCA (1953-1955).
Commandant de la brigade légère 1, de la division 2 (1958-1961), du corps d'armée de campagne 1 (1962-1967).
""".strip()

# Deep usage of the library

**Extraction part**

In [2]:
# Extract the information from the text
information = knex.extraction(text)

# Analyze the extracted information
info_dfs = information.dataframes()
for key in info_dfs.keys():
    print(key)
    display(info_dfs[key])

persons


,name,gender,origins,religion,birth_date,birth_place,death_date,death_place,father_name,mother_name
0,René Dubois,male,Buttes,protestant,1905.08.17,Les Verrières,1976.11.16,Marin-Epagnier,Henri Adolphe Dubois,None
1,Henri Adolphe,None,None,None,None,None,None,None,None,None
2,Suzanne Mina Arnoux,None,Française,None,None,None,None,None,Jules Cyprien,None
3,Jules Cyprien,None,None,None,None,None,None,None,None,None


relationships


,person1_name,person2_name,date_begin,date_end,relationship_type
0,René Dubois,Suzanne Mina Arnoux,None,None,marriage
1,Henri Adolphe,René Dubois,None,None,parental
2,Suzanne Mina Arnoux,René Dubois,None,None,marriage
3,Jules Cyprien,Suzanne Mina Arnoux,None,None,marriage


activities


,person_name,name,place,date_begin,date_end,activity_type,institution,discipline
0,René Dubois,Officier instructeur,None,1929,None,job,None,None
1,René Dubois,Chef d'état-major des troupes d'aviation et de...,None,1953,1955,job,None,None
2,René Dubois,Commandant de la brigade légère 1,None,1958,1961,job,None,None
3,René Dubois,Commandant du corps d'armée de campagne 1,None,1962,1967,job,None,None
4,René Dubois,Licence en sciences économiques,Neuchâtel,None,None,formation,Neuchâtel,None
5,Henri Adolphe,fonctionnaire postal,None,None,None,formation,None,None
6,Suzanne Mina Arnoux,wife of René Dubois,None,None,None,None,None,None
7,Jules Cyprien,None,None,None,None,None,None,None


**Knowledge part**

In [3]:
# Transform the extracted information into a graph
graph = knex.knowledge(information)

# Analyze created entities and triples, and ontology
entities, triples, classes, properties = graph.dataframes()

# Save the dataframes
entities.to_csv('entities.csv', index=False)
triples.to_csv('triples.csv', index=False)
classes.to_csv('classes.csv', index=False)
properties.to_csv('properties.csv', index=False)

# Get the full generated graph in a global dataframe
graph_df = graph.to_dataframe()
graph_df.to_csv( './graph.csv', index=False)

# Visualize the generated graph
graph.get_visuals('./graph.html')

# Save the graph on disk
graph.save('./graph.pkl')

./graph.html
Graph saved to disk


# Full pipeline of the library

In [4]:
# Execute the full pipeline
graph = knex.knowledge_extraction(text)

# Results
graph.to_dataframe().to_csv('./graph.csv', index=False)
graph.get_visuals('./graph.html')
graph.save('./graph.pkl')

./graph.html
Graph saved to disk
